In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 870kB 34.9MB/s 
     |████████████████████████████████| 3.3MB 26.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=a4a1d4fb846509ebacebd771cfd4acc5bea1b888ead37faf02ea8af0c5c57de1
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [14]:

! pip list | grep "tensorflow"   # Check tensorflow==2.0.0, tensorflow-gpu==2.0.0
! pip list | grep "transformers" # Check transformers>=2.0.0

tensorflow                    2.4.1         
tensorflow-datasets           4.0.1         
tensorflow-estimator          2.4.0         
tensorflow-gcs-config         2.4.0         
tensorflow-hub                0.12.0        
tensorflow-metadata           0.29.0        
tensorflow-probability        0.12.1        
transformers                  4.5.1         


In [15]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, glue_convert_examples_to_features,AutoTokenizer
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm.notebook import tqdm
import numpy as np
# import pandas as pd
# import tensorflow as tf

In [16]:
from transformers import TFBertForSequenceClassification,AutoModelForSequenceClassification,AutoConfig
# model = AutoModelForSequenceClassification.from_pretrained('Hate-speech-CNERG/dehatebert-mono-arabic')
# model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-arabic")


# Data Pipeline
*   Loading the datasets from CSVs
*   Preprocessing (Tokenization, Truncation & Padding)
*   Creating efficient data pipelines using tf.data



In [17]:
train_path = '/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/train.xlsx'
valid_path='/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/dev.xlsx'
test_path = '/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx'
test_labels_path = '/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/testLabels.xlsx'
subm_path = '/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/sample_submission.xlsx'

In [1]:
df_train = pd.read_excel(train_path)
df_valid = pd.read_excel(valid_path)
df_test = pd.read_excel(test_path)
df_test_labels = pd.read_excel(test_labels_path)
# s = pd.Series([1, 2, 3, 4])

# df_test_labels = df_test_labels.set_index('id')

df_train = df_train.rename(columns={'off': 'OFF_Labels','hs': 'HS_Labels'})
df_valid = df_valid.rename(columns={'off': 'OFF_Labels','hs': 'HS_Labels'})
df_test_labels=df_test_labels.rename(columns={'label':'HS_Labels'})

label_cols = ['HS_Labels']
df_train.head()



NameError: ignored

In [19]:
bert_model_name = 'Hate-speech-CNERG/dehatebert-mono-arabic'
from transformers import AutoConfig,TFAutoModel, AutoModelForSequenceClassification,BertForSequenceClassification,TFBertForSequenceClassification
# Download configuration from huggingface.co and cache.
# config = AutoConfig.from_pretrained(bert_model_name)
# model = AutoModelForSequenceClassification.from_config(config)

# model = TFBertForSequenceClassification.from_pretrained(bert_model_name)
# model.train()

tokenizer = AutoTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
MAX_LEN = 128

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(
                            sentence,                  # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_seq_len,  # Truncate all sentences.
                            truncation=True,
                    )
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)



df_train = df_train.dropna()
df_valid = df_valid.dropna()
x_train = df_train["tweets"].astype(str).tolist()
x_valid = df_valid["tweets"].astype(str).tolist()
# tweets_encodings = tokenizer(tweets, truncation=True, padding=True)



train_input_ids = tokenize_sentences(x_train, tokenizer, MAX_LEN)
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_attention_masks = create_attention_masks(train_input_ids)


valid_input_ids = tokenize_sentences(x_valid, tokenizer, MAX_LEN)
valid_input_ids = pad_sequences(valid_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
valid_attention_masks = create_attention_masks(valid_input_ids)

In [20]:
# from sklearn.model_selection import train_test_split

y_train =  df_train[label_cols].values
y_valid = df_valid[label_cols].values
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

train_size = len(train_input_ids)
validation_size = len(valid_input_ids)
print('Train input size: ',train_size)
print('Validation input size: ',validation_size)

Train input size:  6984
Validation input size:  1000


In [21]:
BATCH_SIZE = 32
NR_EPOCHS = 1

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_input_ids, train_attention_masks, y_train), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((valid_input_ids, valid_attention_masks, y_valid), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)

In [29]:
!pip install transformers sentencepiece

In [33]:
# model = TFAutoModel.from_pretrained(bert_model_name)

# from transformers import *

model =TFAutoModel.from_pretrained(bert_model_name, from_pt=True)

model.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


AttributeError: ignored

In [30]:
# from transformers import AdamW
# optimizer = AdamW(model.parameters(), lr=1e-5)

In [31]:
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)
model.fit(train_dataset, epochs=2, steps_per_epoch=115)

AttributeError: ignored

# BERT MODEL | 

In [9]:
# from transformers import TFBertModel,TFAutoModel
# from tensorflow.keras.layers import Dense, Flatten

# class BertClassifier(tf.keras.Model):    
#     def __init__(self, bert: TFBertModel, num_classes: int):
#         super().__init__()
#         self.bert = bert
#         self.classifier = Dense(num_classes, activation='sigmoid')
        
#     @tf.function
#     def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
#         outputs = self.bert(input_ids,
#                                attention_mask=attention_mask,
#                                token_type_ids=token_type_ids,
#                                position_ids=position_ids,
#                                head_mask=head_mask)
#         cls_output = outputs[1]
#         cls_output = self.classifier(cls_output)
                
#         return cls_output
# # model = TFAutoModel.from_pretrained("emilyalsentzer/Bio_Discharge_Summary_BERT", from_pt=True)
# model = BertClassifier(TFAutoModel.from_pretrained(bert_model_name, from_pt=True), len(label_cols))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


# Training Loop

In [10]:
# import time
# from transformers import create_optimizer

# steps_per_epoch = train_size // BATCH_SIZE
# validation_steps = validation_size // BATCH_SIZE

# # | Loss Function
# loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# train_loss = tf.keras.metrics.Mean(name='train_loss')
# validation_loss = tf.keras.metrics.Mean(name='test_loss')

# # | Optimizer (with 1-cycle-policy)
# warmup_steps = steps_per_epoch // 3
# total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps
# optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

# # | Metrics
# train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
# validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

# @tf.function
# def train_step(model, token_ids, masks, labels):
#     labels = tf.dtypes.cast(labels, tf.float32)

#     with tf.GradientTape() as tape:
#         predictions = model(token_ids, attention_mask=masks)
#         loss = loss_object(labels, predictions)

#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#     train_loss(loss)

#     for i, auc in enumerate(train_auc_metrics):
#         auc.update_state(labels[:,i], predictions[:,i])



        
# @tf.function
# def validation_step(model, token_ids, masks, labels):
#     labels = tf.dtypes.cast(labels, tf.float32)

#     predictions = model(token_ids, attention_mask=masks, training=False)
#     v_loss = loss_object(labels, predictions)

#     validation_loss(v_loss)
#     for i, auc in enumerate(validation_auc_metrics):
#         auc.update_state(labels[:,i], predictions[:,i])
                                              
# def train(model, train_dataset, val_dataset, train_steps_per_epoch, val_steps_per_epoch, epochs):
#     for epoch in range(epochs):
#         print('=' * 50, f"EPOCH {epoch}", '=' * 50)

#         start = time.time()

#         for i, (token_ids, masks, labels) in enumerate(tqdm(train_dataset, total=train_steps_per_epoch)):
#             train_step(model, token_ids, masks, labels)
#             if i % 1000 == 0:
#                 print(f'\nTrain Step: {i}, Loss: {train_loss.result()}')
#                 for i, label_name in enumerate(label_cols):
#                     print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
#                     train_auc_metrics[i].reset_states()
        
#         for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset, total=val_steps_per_epoch)):
#             validation_step(model, token_ids, masks, labels)

#         print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

#         for i, label_name in enumerate(label_cols):
#             print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
#             validation_auc_metrics[i].reset_states()

#         print('\n')

        
# train(model, train_dataset, validation_dataset, train_steps_per_epoch=steps_per_epoch, val_steps_per_epoch=validation_steps, epochs=NR_EPOCHS)

================================================== EPOCH 0 ==================================================


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

AttributeError: ignored

In [ ]:
print(tf.__version__)

In [32]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model1,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    # eval_dataset=test_dataset            # evaluation dataset
)

In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:

# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# data = tfds.load('glue/mrpc')
# train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
# train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)